In [79]:
import os
import pandas as pd
import numpy as np
import matplotlib as plt
import tensorflow as tf
from tqdm import tqdm

In [80]:
# Import data 
game_state_20 = pd.read_csv('data/full_data_20.csv')
game_state_40 = pd.read_csv('data/full_data_40.csv')
game_state_60 = pd.read_csv('data/full_data_60.csv')
game_state_80 = pd.read_csv('data/full_data_80.csv')
game_state_100 = pd.read_csv('data/full_data_100.csv')

game_states = [game_state_20, game_state_40, game_state_60, game_state_80, game_state_100]

# Check data format and for missing values

#print(game_state_20.dtypes)
#for df in game_states:
#   print(df.shape)
#   print(df.isnull().sum())

# Remove total game time, remaining time, winner, and redundant columns
for df in game_states:
    df.drop(columns=['Unnamed: 0', 'matchID', 'fullTimeMS', 'timePercent', 'redFirstBlood', 'redWin'], inplace=True)
    df['blueFirstBlood'] = df['blueFirstBlood'].astype(int)

In [81]:
# Prepare training, validation, and test datasets for 20% game time

train_frac = 0.8
valid_frac = 0.1

train_set = game_state_20.sample(frac = 0.8)
remaining_set = game_state_20.drop(train_set.index)
valid_set = remaining_set.sample(frac = valid_frac / (1 - train_frac))
test_set = remaining_set.drop(valid_set.index)

# Separate wins
train_wins = train_set.pop('blueWin')
valid_wins = valid_set.pop('blueWin')
test_wins = test_set.pop('blueWin')

print(game_state_20.shape, train_set.shape, valid_set.shape, test_set.shape)


(64556, 36) (51645, 35) (6456, 35) (6455, 35)


In [108]:
### Try simple DNN classification

# Convert dataframes to tensors
train_data = tf.convert_to_tensor(train_set)
train_labels = tf.convert_to_tensor(train_wins)

valid_data = tf.convert_to_tensor(valid_set)
valid_labels = tf.convert_to_tensor(valid_wins)

#train_data = train_data[..., tf.newaxis]
#valid_data = valid_data[..., tf.newaxis]
# Define batches

def get_batch(data, labels, batch_size):
    return tf.data.Dataset.from_tensor_slices((data, labels)).batch(batch_size)

train_batched = get_batch(train_data, train_labels, 64)
valid_batched = get_batch(valid_data, valid_labels, 64)

print(tf.shape(train_data), tf.shape(train_labels))


tf.Tensor([51645    35], shape=(2,), dtype=int32) tf.Tensor([51645], shape=(1,), dtype=int32)


In [119]:
# Define model
class DNNModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        # Single dense layer
        self.layer1 = tf.keras.layers.Dense(35, activation='relu')
        self.layer2 = tf.keras.layers.Dense(35, activation='relu')
        self.layer3 = tf.keras.layers.Dense(35, activation='relu')
        self.layer4 = tf.keras.layers.Dense(35, activation='relu')
        self.layer5 = tf.keras.layers.Dense(1)
        
    def call(self, input):
        input = self.layer1(input)
        input = self.layer2(input)
        input = self.layer3(input)
        input = self.layer4(input)
        return self.layer5(input)

model = DNNModel()

# Choose optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss_func = tf.keras.losses.CategoricalCrossentropy()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

valid_loss = tf.keras.metrics.Mean(name='validation_loss')
valid_accuracy = tf.keras.metrics.CategoricalAccuracy(name='validation_accuracy')


# Define training step
@tf.function
def train_step(match_states, outcomes):
    with tf.GradientTape() as tape:
        
        predictions = tf.squeeze(model(match_states))
        loss = loss_func(outcomes, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(outcomes, predictions)

# Define testing step
@tf.function
def test_step(match_states, outcomes):
    predictions = tf.squeeze(model(match_states))
    loss = loss_func(outcomes, predictions)

    valid_loss(loss)
    valid_accuracy(outcomes, predictions)

# Set output
output_dir = './output'
output_prefix = os.path.join(output_dir, "DNN_2l_20")


# Run
EPOCHS = 100

for epoch in tqdm(range(EPOCHS)):
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()

    for match_states, outcomes in train_batched:
        train_step(match_states, outcomes)

    for match_states, outcomes in valid_batched:
        test_step(match_states, outcomes)

    if epoch % 10 == 0:
        model.save_weights(output_prefix)

    print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result()}, '
    f'Validation Loss: {valid_loss.result()}, '
    f'Validation Accuracy: {valid_accuracy.result()}'
  )

model.save_weights(output_prefix)

  1%|▋                                                                   | 1/100 [00:02<03:26,  2.09s/it]

Epoch 1, Loss: 134.55580139160156, Accuracy: 0.01610904559493065, Validation Loss: 135.95675659179688, Validation Accuracy: 0.009900989942252636


  2%|█▎                                                                  | 2/100 [00:03<02:19,  1.42s/it]

Epoch 2, Loss: 133.5020751953125, Accuracy: 0.022304832935333252, Validation Loss: 135.41644287109375, Validation Accuracy: 0.009900989942252636


  3%|██                                                                  | 3/100 [00:04<01:57,  1.21s/it]

...

Epoch 98, Loss: 132.41775512695312, Accuracy: 0.0037174720782786608, Validation Loss: 134.94427490234375, Validation Accuracy: 0.019801979884505272


 99%|██████████████████████████████████████████████████████████████████▎| 99/100 [01:39<00:00,  1.08it/s]

Epoch 99, Loss: 132.39999389648438, Accuracy: 0.002478314796462655, Validation Loss: 134.8787841796875, Validation Accuracy: 0.009900989942252636


100%|██████████████████████████████████████████████████████████████████| 100/100 [01:40<00:00,  1.00s/it]

Epoch 100, Loss: 132.37986755371094, Accuracy: 0.002478314796462655, Validation Loss: 134.8252410888672, Validation Accuracy: 0.009900989942252636
